In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import re
from collections import Counter
import pickle

# ============================================
# SMAPE METRIC
# ============================================
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    smape_val = np.mean(diff / (denominator + 1e-10)) * 100
    return smape_val

class SMAPELoss(nn.Module):
    def __init__(self):
        super(SMAPELoss, self).__init__()
    
    def forward(self, y_pred, y_true):
        denominator = (torch.abs(y_true) + torch.abs(y_pred)) / 2.0
        diff = torch.abs(y_true - y_pred)
        smape_loss = torch.mean(diff / (denominator + 1e-10)) * 100
        return smape_loss

# ============================================
# DATA PREPROCESSING
# ============================================
class TextPreprocessor:
    def __init__(self, vocab_size=10000, max_length=50):
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        
    def clean_text(self, text):
        text = str(text).lower()
        text = re.sub(r'[^a-z0-9\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def build_vocab(self, texts):
        all_words = [word for text in texts for word in self.clean_text(text).split()]
        word_freq = Counter(all_words)
        most_common = word_freq.most_common(self.vocab_size - 2)
        
        for idx, (word, _) in enumerate(most_common, start=2):
            self.word2idx[word] = idx
            self.idx2word[idx] = word
            
        print(f"Vocabulary size: {len(self.word2idx)}")
        return self
    
    def text_to_sequence(self, text):
        cleaned = self.clean_text(text)
        words = cleaned.split()
        sequence = [self.word2idx.get(word, 1) for word in words]
        
        if len(sequence) < self.max_length:
            sequence += [0] * (self.max_length - len(sequence))
        else:
            sequence = sequence[:self.max_length]
        return sequence
    
    def texts_to_sequences(self, texts):
        return [self.text_to_sequence(text) for text in texts]
    
    def save(self, filepath):
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)
    
    @staticmethod
    def load(filepath):
        with open(filepath, 'rb') as f:
            return pickle.load(f)

# ============================================
# DATASET CLASS
# ============================================
class PriceDataset(Dataset):
    def __init__(self, sequences, prices=None):
        self.sequences = torch.LongTensor(sequences)
        self.prices = torch.FloatTensor(prices) if prices is not None else None
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        if self.prices is not None:
            return self.sequences[idx], self.prices[idx]
        else:
            return self.sequences[idx]

# ============================================
# MODEL 1: Enhanced Bi-LSTM with Multi-Head Attention
# ============================================
class BiLSTMWithAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, 
                 num_layers=3, dropout=0.3, num_heads=4):
        super(BiLSTMWithAttention, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.2)
        
        # Bi-LSTM layers
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_dim, 
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
            bidirectional=True
        )
        
        # Multi-head attention
        self.num_heads = num_heads
        self.head_dim = hidden_dim * 2 // num_heads
        
        self.query = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.key = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.value = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        
        # Output layers
        self.fc1 = nn.Linear(hidden_dim * 2, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(dropout)
        
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(dropout)
        
        self.fc4 = nn.Linear(64, 1)
        
        self.relu = nn.ReLU()
        
    def multi_head_attention(self, lstm_out):
        batch_size, seq_len, hidden_size = lstm_out.size()
        
        # Linear projections
        Q = self.query(lstm_out).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(lstm_out).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(lstm_out).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attention_weights = torch.softmax(scores, dim=-1)
        
        # Apply attention to values
        attended = torch.matmul(attention_weights, V)
        
        # Concatenate heads
        attended = attended.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_size)
        
        # Pool over sequence
        context_vector = torch.mean(attended, dim=1)
        
        return context_vector
    
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        embedded = self.embedding_dropout(embedded)
        
        # Bi-LSTM
        lstm_out, _ = self.lstm(embedded)
        
        # Multi-head attention
        context_vector = self.multi_head_attention(lstm_out)
        
        # Fully connected layers with batch norm
        x = self.relu(self.bn1(self.fc1(context_vector)))
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        
        x = self.fc4(x)
        
        return x.squeeze(-1)

# ============================================
# MODEL 2: CNN-LSTM Hybrid
# ============================================
class CNNLSTMHybrid(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, 
                 num_layers=2, dropout=0.3):
        super(CNNLSTMHybrid, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.2)
        
        # Multiple CNN filters with different kernel sizes
        self.conv1 = nn.Conv1d(embedding_dim, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(embedding_dim, 128, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(embedding_dim, 128, kernel_size=5, padding=2)
        
        self.bn_conv1 = nn.BatchNorm1d(128)
        self.bn_conv2 = nn.BatchNorm1d(128)
        self.bn_conv3 = nn.BatchNorm1d(128)
        
        # Pool CNN outputs
        self.pool = nn.AdaptiveMaxPool1d(1)
        
        # Bi-LSTM on CNN features
        self.lstm = nn.LSTM(
            384,  # 128 * 3 from three CNN filters
            hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
            bidirectional=True
        )
        
        # Attention
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * 2 + 384, 256)  # Concatenate LSTM + CNN features
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(dropout)
        
        self.fc3 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(dropout)
        
        self.fc4 = nn.Linear(64, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)  # (batch, seq_len, embedding_dim)
        embedded = self.embedding_dropout(embedded)
        
        # CNN expects (batch, channels, length)
        embedded_transposed = embedded.transpose(1, 2)  # (batch, embedding_dim, seq_len)
        
        # Apply multiple CNN filters
        conv1_out = self.relu(self.bn_conv1(self.conv1(embedded_transposed)))
        conv2_out = self.relu(self.bn_conv2(self.conv2(embedded_transposed)))
        conv3_out = self.relu(self.bn_conv3(self.conv3(embedded_transposed)))
        
        # Concatenate CNN outputs
        conv_out = torch.cat([conv1_out, conv2_out, conv3_out], dim=1)  # (batch, 384, seq_len)
        
        # Global max pooling for CNN features
        cnn_features = self.pool(conv_out).squeeze(-1)  # (batch, 384)
        
        # Transpose back for LSTM
        conv_out_transposed = conv_out.transpose(1, 2)  # (batch, seq_len, 384)
        
        # Bi-LSTM
        lstm_out, _ = self.lstm(conv_out_transposed)
        
        # Attention on LSTM output
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        lstm_context = torch.sum(attention_weights * lstm_out, dim=1)
        
        # Concatenate CNN and LSTM features
        combined = torch.cat([cnn_features, lstm_context], dim=1)
        
        # Fully connected layers
        x = self.relu(self.bn1(self.fc1(combined)))
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        
        x = self.fc4(x)
        
        return x.squeeze(-1)

# ============================================
# MODEL 3: Advanced CNN-BiLSTM with Residual Connections
# ============================================
class CNNBiLSTMResidual(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, 
                 num_layers=2, dropout=0.3):
        super(CNNBiLSTMResidual, self).__init__()
        
        # Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.2)
        
        # CNN Block 1
        self.conv1 = nn.Conv1d(embedding_dim, 128, kernel_size=3, padding=1)
        self.bn_conv1 = nn.BatchNorm1d(128)
        
        # CNN Block 2 (residual)
        self.conv2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.bn_conv2 = nn.BatchNorm1d(128)
        
        # CNN Block 3 (residual)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn_conv3 = nn.BatchNorm1d(256)
        
        # Bi-LSTM
        self.lstm = nn.LSTM(
            256,
            hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
            bidirectional=True
        )
        
        # Self-attention
        self.attention = nn.MultiheadAttention(hidden_dim * 2, num_heads=4, dropout=dropout, batch_first=True)
        
        # Output layers
        self.fc1 = nn.Linear(hidden_dim * 2, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(dropout)
        
        self.fc3 = nn.Linear(128, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        embedded = self.embedding_dropout(embedded)
        
        # CNN blocks with residual connections
        x_cnn = embedded.transpose(1, 2)  # (batch, embedding_dim, seq_len)
        
        # Block 1
        x_cnn = self.relu(self.bn_conv1(self.conv1(x_cnn)))
        
        # Block 2 (with residual)
        residual = x_cnn
        x_cnn = self.relu(self.bn_conv2(self.conv2(x_cnn)))
        x_cnn = x_cnn + residual  # Residual connection
        
        # Block 3
        x_cnn = self.relu(self.bn_conv3(self.conv3(x_cnn)))
        
        # Back to sequence format for LSTM
        x_cnn = x_cnn.transpose(1, 2)  # (batch, seq_len, 256)
        
        # Bi-LSTM
        lstm_out, _ = self.lstm(x_cnn)
        
        # Self-attention
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Pool over sequence
        pooled = torch.mean(attn_out, dim=1)
        
        # Fully connected layers
        x = self.relu(self.bn1(self.fc1(pooled)))
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = self.fc3(x)
        
        return x.squeeze(-1)

# ============================================
# TRAINING PIPELINE
# ============================================
class PricePredictionTrainer:
    def __init__(self, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.best_smape = float('inf')
        print(f"Using device: {device}")
        
    def train(self, train_loader, val_loader, epochs=100, lr=0.001, patience=15):
        criterion = SMAPELoss()
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=lr, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', patience=5, factor=0.5, verbose=True
        )
        
        patience_counter = 0
        
        for epoch in range(epochs):
            # Training
            self.model.train()
            train_loss = 0
            for sequences, prices in train_loader:
                sequences, prices = sequences.to(self.device), prices.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(sequences)
                loss = criterion(outputs, prices)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                train_loss += loss.item()
            
            train_loss /= len(train_loader)
            
            # Validation
            val_loss, val_smape = self._validate(val_loader, criterion)
            scheduler.step(val_smape)
            
            # Save best model
            if val_smape < self.best_smape:
                self.best_smape = val_smape
                torch.save({'model_state_dict': self.model.state_dict()}, '/kaggle/input/amazon-ml-challenge-2025/best_model.pth')
                patience_counter = 0
                print(f'Epoch [{epoch+1}/{epochs}] ⭐ Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val SMAPE: {val_smape:.4f}')
            else:
                patience_counter += 1
                if (epoch + 1) % 5 == 0:
                    print(f'Epoch [{epoch+1}/{epochs}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val SMAPE: {val_smape:.4f}')
            
            # Early stopping
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        # Load best model
        checkpoint = torch.load('/kaggle/input/amazon-ml-challenge-2025/best_model.pth')
        self.model.load_state_dict(checkpoint['model_state_dict'])
        print(f"\n🎯 Best validation SMAPE: {self.best_smape:.4f}")
    
    def _validate(self, val_loader, criterion):
        self.model.eval()
        val_loss = 0
        all_preds, all_targets = [], []
        
        with torch.no_grad():
            for sequences, prices in val_loader:
                sequences, prices = sequences.to(self.device), prices.to(self.device)
                outputs = self.model(sequences)
                val_loss += criterion(outputs, prices).item()
                all_preds.extend(outputs.cpu().numpy())
                all_targets.extend(prices.cpu().numpy())
        
        return val_loss / len(val_loader), smape(np.array(all_targets), np.array(all_preds))

    def predict(self, test_loader):
        self.model.eval()
        all_preds = []
        with torch.no_grad():
            for sequences in test_loader:
                sequences = sequences.to(self.device)
                outputs = self.model(sequences)
                all_preds.extend(outputs.cpu().numpy())
        return np.array(all_preds)

# ============================================
# MAIN PIPELINE FUNCTIONS
# ============================================
def train_pipeline(train_df, model_type='bilstm', val_split=0.2):
    """
    model_type: 'bilstm', 'cnn_lstm', or 'cnn_bilstm_res'
    """
    print("="*50 + f"\nTRAINING PIPELINE - {model_type.upper()}\n" + "="*50)
    
    # Prepare data
    print("\n1. Preparing data...")
    train_df = train_df.dropna(subset=['catalog_content', 'price'])
    descriptions, prices = train_df['catalog_content'].values, train_df['price'].values
    
    X_train, X_val, y_train, y_val = train_test_split(descriptions, prices, test_size=val_split, random_state=42)
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")
    
    # Build vocabulary
    print("\n2. Building vocabulary...")
    preprocessor = TextPreprocessor(vocab_size=15000, max_length=100)
    preprocessor.build_vocab(X_train)
    
    X_train_seq = preprocessor.texts_to_sequences(X_train)
    X_val_seq = preprocessor.texts_to_sequences(X_val)
    
    train_loader = DataLoader(PriceDataset(X_train_seq, y_train), batch_size=128, shuffle=True)
    val_loader = DataLoader(PriceDataset(X_val_seq, y_val), batch_size=128)
    
    # Initialize model
    print(f"\n3. Initializing {model_type.upper()} model...")
    vocab_size = len(preprocessor.word2idx)
    
    if model_type == 'bilstm':
        model = BiLSTMWithAttention(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=3)
    elif model_type == 'cnn_lstm':
        model = CNNLSTMHybrid(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=2)
    elif model_type == 'cnn_bilstm_res':
        model = CNNBiLSTMResidual(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=2)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")
    
    # Train
    print("\n4. Training model...")
    trainer = PricePredictionTrainer(model)
    trainer.train(train_loader, val_loader, epochs=100, lr=0.001, patience=15)
    
    preprocessor.save('preprocessor.pkl')
    print("\n✅ Preprocessor and best model saved.")
    
def prediction_pipeline(test_df, model_type='bilstm', 
                       preprocessor_path='preprocessor.pkl', 
                       model_path='/kaggle/input/amazon-ml-challenge-2025/best_model.pth'):
    print("="*50 + f"\nPREDICTION PIPELINE - {model_type.upper()}\n" + "="*50)
    
    # Load preprocessor
    print("\n1. Loading preprocessor and model...")
    preprocessor = TextPreprocessor.load(preprocessor_path)
    
    # Prepare test data
    test_descriptions = test_df['catalog_content'].values
    test_sequences = preprocessor.texts_to_sequences(test_descriptions)
    test_loader = DataLoader(PriceDataset(test_sequences), batch_size=128)
    
    # Load model
    vocab_size = len(preprocessor.word2idx)
    
    if model_type == 'bilstm':
        model = BiLSTMWithAttention(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=3)
    elif model_type == 'cnn_lstm':
        model = CNNLSTMHybrid(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=2)
    elif model_type == 'cnn_bilstm_res':
        model = CNNBiLSTMResidual(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=2)
    
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Make predictions
    print("\n2. Making predictions...")
    trainer = PricePredictionTrainer(model)
    predictions = trainer.predict(test_loader)
    
    # Create submission
    submission_df = pd.DataFrame({'sample_id': test_df['sample_id'], 'price': predictions})
    submission_df.to_csv('submission.csv', index=False)
    print("\n✅ Predictions saved to 'submission.csv'")

# ============================================
# USAGE EXAMPLE
# ============================================
if __name__ == "__main__":
    # Load data
    train_df = pd.read_csv('/kaggle/input/amazon-ml-challenge-2025/train.csv')
    
    # Choose one model:
    
    # Option 1: Bi-LSTM with Multi-Head Attention (Best for long sequences)
    train_pipeline(train_df, model_type='bilstm')
    
    # Option 2: CNN-LSTM Hybrid (Best for local patterns + sequences)
    # train_pipeline(train_df, model_type='cnn_lstm')
    
    # Option 3: CNN-BiLSTM with Residual (Most advanced)
    # train_pipeline(train_df, model_type='cnn_bilstm_res')
    
    # Predict
    test_df = pd.read_csv('/kaggle/input/amazon-ml-challenge-2025/test.csv')
    prediction_pipeline(test_df, model_type='bilstm')  # Match the model type used in training

In [ ]:
# ============================================
# FINE-TUNING PIPELINE (Add this new function)
# ============================================
def finetune_pipeline(train_df,
                      preprocessor_path='preprocessor.pkl',
                      model_path='best_model.pth',
                      val_split=0.2):
    """
    Loads a pre-trained model and its preprocessor to continue training (fine-tune).
    
    Args:
        train_df (pd.DataFrame): DataFrame with data for fine-tuning.
        preprocessor_path (str): Path to the saved preprocessor from initial training.
        model_path (str): Path to the saved pre-trained model (.pth).
        val_split (float): The proportion of the dataset to use for validation.
    """
    print("="*50 + "\nFINE-TUNING PIPELINE\n" + "="*50)
    
    # --- 1. Data Loading and Cleaning (same as before) ---
    train_df = train_df.dropna(subset=['catalog_content', 'price'])
    train_df['price'] = pd.to_numeric(train_df['price'], errors='coerce')
    train_df = train_df.dropna(subset=['price'])
    
    descriptions, prices = train_df['catalog_content'].values, train_df['price'].values
    X_train, X_val, y_train, y_val = train_test_split(descriptions, prices, test_size=val_split, random_state=42)
    print(f"Fine-tuning with {len(X_train)} samples, Validating with {len(X_val)} samples.")

    # --- 2. KEY CHANGE: Load the existing preprocessor ---
    # We DO NOT build a new vocabulary. We must use the one the model already knows.
    print(f"Loading preprocessor from {preprocessor_path}...")
    preprocessor = TextPreprocessor.load(preprocessor_path)
    
    # Process text using the loaded preprocessor
    X_train_seq = preprocessor.texts_to_sequences(X_train)
    X_val_seq = preprocessor.texts_to_sequences(X_val)
    
    train_loader = DataLoader(PriceDataset(X_train_seq, y_train), batch_size=128, shuffle=True)
    val_loader = DataLoader(PriceDataset(X_val_seq, y_val), batch_size=128)

    # --- 3. KEY CHANGE: Instantiate model and load pre-trained weights ---
    vocab_size = len(preprocessor.word2idx)
    model = BiLSTMWithAttention(vocab_size, embedding_dim=128, hidden_dim=256, num_layers=3)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Loading model weights from {model_path}...")
    checkpoint = torch.load(model_path, map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])
    print("Model weights loaded successfully.")
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")

    # --- 4. Training with a lower learning rate ---
    # The trainer class can be reused as is.
    trainer = PricePredictionTrainer(model)
    
    # KEY CHANGE: Use a smaller learning rate for fine-tuning to make small adjustments.
    print("Starting fine-tuning process with a lower learning rate...")
    # Note the much smaller learning rate (lr)
    trainer.train(train_loader, val_loader, epochs=50, lr=5e-5, patience=10)
    
    # --- 5. Save the fine-tuned model to a NEW file ---
    # This avoids overwriting your original best model.
    trainer.model.save_state_dict('best_model.pth')
    print("\n✅ Fine-tuning complete. New model saved to 'best_model.pth'.")

In [ ]:
# ============================================
# USAGE EXAMPLE
# ============================================
if _name_ == "_main_":
    # You can fine-tune on the same training data or a new, similar dataset.
    # Here, we use the original training data.
    train_df = pd.read_csv('train.csv')

    # Run the new fine-tuning function
    finetune_pipeline(train_df)
    
    # You can now use 'finetuned_model.pth' for predictions
    # test_df = pd.read_csv('test.csv')
    # prediction_pipeline(test_df, model_path='finetuned_model.pth')